# Exploring the Data
This is using NY data. Looks like inside aribnb data is same format so it can be used for other cities


In [2]:
import os
import io
import re
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
#read data, specify index so it's easier to join and search using loc
path = '../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'listings.csv')

listings = pd.read_csv(listings_csv, index_col = 'id')

In [9]:
#shape
total_row = listings.shape[0]
listings.shape

(50796, 105)

In [5]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

In [41]:
def less_than_50_percent(colname):
    isnull_count = listings[colname].isna().sum()
    if isnull_count/total_row > .5:
        removal_reason = colname
        return True


In [45]:
columns = list(listings)
remove_columns = []
for column in columns:
    remove_column_y_n = less_than_50_percent(column)
    if remove_column_y_n:
        remove_columns.append(column)

print(remove_columns)

['notes', 'thumbnail_url', 'medium_url', 'xl_picture_url', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names']
